In [1]:
from fastai.structured import *
from fastai.column_data import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# lets load the data
df = pd.read_csv('course_ratings.csv')

In [3]:
#lets remove high rating rows
joined = df.drop(df[df['rating'] > 5].index)

In [4]:
# lets define catigorical vars
cat_vars = ['user','course','category','job','institution','state']
#coninuous vars
cont_vars = ['rating']

# df length
n = len(df)

# test percentage
testp = n*.2

#lets update features, categoricial should be category and continuous should be float32
for v in cat_vars: 
    joined[v] = joined[v].astype('category').cat.as_ordered()
    
for v in cont_vars:
    joined[v] = joined[v].fillna(0).astype('float32')

#create a test batch
joined, joined_test = train_test_split(joined, test_size=0.25)

In [5]:
# helper function to covert the categorical variables into continguous ones that we can plug into a matrix
df, y, nas = proc_df(joined, 'rating', do_scale=False)
df_test,_,_ = proc_df(joined_test, 'rating', do_scale=False)

In [6]:
# create a validation set of ids
train_ratio = 0.75
train_size = int(len(df) * train_ratio)
val_idx = list(range(train_size, len(df)))

In [7]:
# now we begin building the model

In [8]:
md = ColumnarModelData.from_data_frame("models/", val_idx, df, y.astype(np.float32), cat_flds=cat_vars, bs=128, test_df=df_test)

In [16]:
#build embedding matrix
cat_sz = [(c, len(joined[c].cat.categories)+1) for c in cat_vars]
#emb_szs = [(39482,50),(209,210),(13,14),(42,42),(13,14),(59,60)]
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]
emb_szs

[(52118, 50), (220, 50), (15, 8), (46, 23), (16, 8), (61, 31)]

In [22]:
#build the learner
m = md.get_learner(emb_szs,0 ,0.04, 1, [1000,500], [0.001,0.01], y_range=(0,5))

In [21]:
m.summary()

TypeError: torch.index_select received an invalid combination of arguments - got ([32;1mtorch.cuda.FloatTensor[0m, [32;1mint[0m, [31;1mtorch.cuda.FloatTensor[0m), but expected (torch.cuda.FloatTensor source, int dim, torch.cuda.LongTensor index)